In [1]:
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
# Necessary for Mac
!which chromedriver


# Mac executable path
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

# Windows executable path(comment out '!which chromedrive' to make it work)
# executable_path = {'executable_path': 'chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [3]:
# define url and visit
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

<h1>NASA Mars News</h1>

In [4]:

# html parser boiler plate
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# have to dig into article before pulling news title to avoid conflict in navbar
articles = soup.find('div', class_='image_and_description_container')
news_title = articles.find('div', class_='content_title').get_text()
news_p = articles.find('div', class_='article_teaser_body').get_text()

print(news_title)

Join NASA for the Launch of the Mars 2020 Perseverance Rover


<h1>JPL Mars Space Images - Featured Image</h1>

In [5]:

# define url and visit
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [6]:

# html parser boiler plate
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
base_url= 'https://www.jpl.nasa.gov'

# have to click image to see source
browser.click_link_by_partial_text('FULL IMAGE')


In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image = soup.find('div', id='fancybox-lock')
src_url = image.find('img')
print(src_url['src'])

/spaceimages/images/mediumsize/PIA18846_ip.jpg


<h1>Mars Weather</h1>

In [8]:

# define url and visit
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [9]:
# html parser boiler plate
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

tweets = soup.find_all('div',lang="en",dir='auto')

# tweet = tweets[2].find('span').get_text()
# print(tweet)

# loop through recent tweets to find the first weather tweet
for tweet in tweets:
    output = tweet.find('span').get_text()
    if 'InSight' in output:
        print(output)
        mars_weather = output
        break


InSight sol 566 (2020-06-29) low -88.4ºC (-127.1ºF) high -4.2ºC (24.4ºF)
winds from the WNW at 5.1 m/s (11.5 mph) gusting to 16.8 m/s (37.6 mph)
pressure at 7.70 hPa


<h1>Mars Facts</h1>

In [10]:

# define url and visit
url = 'https://space-facts.com/mars/'
browser.visit(url)

#  html parser boiler plate
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [11]:

main = soup.find('table', id='tablepress-p-mars-no-2')
main

<table class="tablepress tablepress-id-p-mars" id="tablepress-p-mars-no-2"><tbody><tr class="row-1 odd"><td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × 10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr class="row-5 odd"><td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/></td></tr><tr class="row-7 odd"><t

<h1>Mars Hemispheres</h1>

In [38]:
# define url and visit
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

#  html parser boiler plate
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
og_url= 'https://astrogeology.usgs.gov/'

# /cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
# 

In [43]:
images = soup.find_all('div', class_='item')
image_url_list = []
for image in images:
    # declare  html parser(for secomnd time going through loop and beyond)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # get url to main image
    thumb_url = image.find('a')['href']
    
    # set image title for image_url_list
    image_title = image.find('h3').get_text()
    
    # visit site for main image and restart html parser
    browser.visit(og_url+thumb_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # grab full image url
    soup.find('img', class_='wide-image').get_text
    print(soup)

    # add title and image url to image_url_list

    # go back to main page
    browser.back()

    


# images[1].find('img')